<a href="https://colab.research.google.com/github/MohitSoni11/cricket-research-cornell/blob/main/1_SSIM_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SSIM Algorithm

**Objective:** Create an algorithm to find extremely similar frames from the frames of a video using the Structural Similarity Index (SSIM).

**Purpose:** We will use this algorithm to filter out extremely similar frames from videos as those frames will be unnecessary data.

### SSIM Background Information

The Structural Similarity Index (SSIM) is a newer equation compared to the MSE, both used for receiving a quantitative measure of the similarity between two images. To compute the SSIM between two images, the images must have the same size and ratio. The SSIM is based on the computation of three factors: luminance ($l$), contrast ($c$), and structure ($s$).

$$SSIM(x, y) = [l(x, y)]^a * [c(x, y)]^{\beta} * [s(x, y)]^{\gamma}$$

where:

$$l(x, y) = \frac{2\mu_x\mu_y + C_1}{\mu_x^2 + \mu_y^2 + C1},$$

$$c(x, y) = \frac{2\sigma_x\sigma_y + C_2}{\sigma_x^2 + \sigma_y^2 + C2},$$

$$s(x, y) = \frac{\sigma_xy + C_3}{\sigma_x\sigma_y + C_3}$$

where $\mu_x, \mu_y, \sigma_x, \sigma_y$ and $\sigma_{xy}$ are the local means, standard deviations, and cross-covariance for images $x, y$. The SSIM values range between 0 to 1 where 1 means a perfect match between the first and second image.

Citation: https://code.adonline.id.au/structural-similarity-index-ssim-in-python/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/My Drive/Colab Notebooks/Cricket-CV-Research/data-compressed.zip'

Archive:  /content/drive/My Drive/Colab Notebooks/Cricket-CV-Research/data-compressed.zip
  inflating: data/basketball-video.mp4  
  inflating: data/batting-practice.mp4  
  inflating: data/black-car.jpg      
  inflating: data/hot-air-balloons.jpg  
  inflating: data/mohit-cricket.mp4  
  inflating: data/virat-kohli-nets.mp4  


In [ ]:
import cv2

# Getting images from drive
car_image = cv2.imread('data/black-car.jpg')
balloons_image = cv2.imread('data/hot-air-balloons.jpg')

In [ ]:
from google.colab.patches import cv2_imshow

# Outputting the images
cv2_imshow(car_image)
cv2_imshow(balloons_image)

## Convert images to grayscale

In [ ]:
car_gray = cv2.cvtColor(car_image, cv2.COLOR_BGR2GRAY)
balloons_gray = cv2.cvtColor(balloons_image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Output images converted to grayscale
cv2_imshow(car_gray)
cv2_imshow(balloons_gray)

## Check dimensions of images

To find the SSIM index between two images, both images must have the same dimensions.

In [ ]:
car_height, car_width = car_gray.shape
balloons_height, balloons_width = balloons_gray.shape

car_ratio = car_height / car_width
balloons_ratio = balloons_height / balloons_width

print('Car Image Dimensions: (' + str(car_width) + ', ' + str(car_height) + ')')
print('Hot Air Balloon Image Dimensions: (' + str(balloons_width) + ', ' + str(balloons_height) + ')')

Car Image Dimensions: (3456, 4608)
Hot Air Balloon Image Dimensions: (3456, 4608)


## Find SSIM for images

In [ ]:
from skimage.metrics import structural_similarity as ssim
print('SSIM:', ssim(car_gray, balloons_gray))

SSIM: 0.13252569624141822


## Automate finding SSIM between any two images

In [4]:
import cv2
from skimage.metrics import structural_similarity as ssim

def find_ssim(image_1, image_2):
  '''
  Finds the SSIM between images `image_1` and `image_2`.
  '''

  # Convert images to grayscale

  gray_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
  gray_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)

  # Make sure that both images have the same size and ratio.

  height_1, width_1 = gray_1.shape
  height_2, width_2 = gray_2.shape

  ratio_1 = height_1 / width_1
  ratio_2 = height_2 / width_2

  # Confirm equal dimensions of both images
  if (round(ratio_1, 2) != round(ratio_2, 2)):
    print('Error: Images are not of the same dimension.')
    exit()

  # Resize first image if it is bigger
  elif height_1 > height_2 and width_1 > width_2:
    print('First image is bigger than second. Resizing first image...')
    image_1 = cv2.resize(gray_1, (width_2, height_2))

  # Resize second image if it is bigger
  elif height_1 < height_2 and width_1 < width_2:
    print('Second image is bigger than first. Resizing second image...')
    image_2 = cv2.resize(gray_2, (width_1, height_1))

  return ssim(gray_1, gray_2)

In [ ]:
print('SSIM:', find_ssim(car_image, balloons_image))

SSIM: 0.13252569624141822


## Import video from data folder

In [ ]:
video = cv2.VideoCapture('data/basketball-video.mp4')

# Printing out video metadata
print('FPS =', video.get(cv2.CAP_PROP_FPS))
print('Total Number of Frames=', video.get(cv2.CAP_PROP_FRAME_COUNT))

FPS = 25.0
Total Number of Frames= 505.0


## Loop through video frames and store SSIM values

Note that we are storing the time of the second frame in the frame pair (instead of the first) because whenever any fluctuation happens in the SSIM graph, the change occurs at the time of the second frame in the frame pair (as that is the frame that changes resulting in lower SSIM values), not the first.

In [ ]:
frame_num = 1
ssim_values = []
frame_times = []
frames_being_compared = []
_, prev_image = video.read()

# Want to loop until frame_num is 504 since loop has 505 frames and we are
# comparing the current frame to the next frame
while frame_num < video.get(cv2.CAP_PROP_FRAME_COUNT):
  _, curr_image = video.read()

  ssim_values.append(find_ssim(prev_image, curr_image))
  prev_image = curr_image

  # Storing the frame pair
  frame_pair = '(' + str(frame_num) + ', ' + str(frame_num + 1) + ')'
  frames_being_compared.append(frame_pair)

  # Storing the time of the second frame (current frame) in the frame pair
  frame_times.append((frame_num + 1) / video.get(cv2.CAP_PROP_FPS));

  if (frame_num < 7) or (frame_num > video.get(cv2.CAP_PROP_FRAME_COUNT) - 5) or (frame_num % 100 == 0):
    print('Stored SSIM for frame pair', frame_pair)

  frame_num += 1

print('Execution completed.')

Stored SSIM for frame pair (1, 2)
Stored SSIM for frame pair (2, 3)
Stored SSIM for frame pair (3, 4)
Stored SSIM for frame pair (4, 5)
Stored SSIM for frame pair (5, 6)
Stored SSIM for frame pair (6, 7)
Stored SSIM for frame pair (100, 101)
Stored SSIM for frame pair (200, 201)
Stored SSIM for frame pair (300, 301)
Stored SSIM for frame pair (400, 401)
Stored SSIM for frame pair (500, 501)
Stored SSIM for frame pair (501, 502)
Stored SSIM for frame pair (502, 503)
Stored SSIM for frame pair (503, 504)
Stored SSIM for frame pair (504, 505)
Execution completed.


In [ ]:
len(ssim_values)

504

In [ ]:
frames_being_compared[0:5]

['(1, 2)', '(2, 3)', '(3, 4)', '(4, 5)', '(5, 6)']

In [ ]:
frame_times[0:5]

[0.08, 0.12, 0.16, 0.2, 0.24]

## Graph SSIM values using external library Plotly

In [ ]:
!pip install plotly==5.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
import plotly.express as px

plot1 = px.line(x=frames_being_compared, y=ssim_values)
plot2 = px.line(x=frame_times, y=ssim_values)
plot3 = px.histogram(x=frame_times, y=ssim_values, nbins = 200)

plot1.update_layout(xaxis=dict(title_text='Frame Pairs - (Frame 1, Frame 2)'),
                    yaxis=dict(title_text='SSIM Metric', range=list([0, 1])))

plot2.update_layout(xaxis=dict(dtick=1, title_text='Time of Second Frame in Frame Pair'),
                    yaxis=dict(title_text='SSIM Metric', range=list([0, 1])))

plot3.update_layout(xaxis=dict(dtick=1, title_text='Time of Second Frame in Frame Pair'),
                    yaxis=dict(title_text='SSIM Metric Sum'))

plot1.show()
plot2.show()
plot3.show()

# Final Program: SSIM Algorithm

In [5]:
import cv2
import plotly.express as px
from skimage.metrics import structural_similarity as ssim

def ssim_graphs(video):
  '''
  Returns 3 graphs detailing the SSIM metric for all possible frame pairs in video `video`.
  '''

  # Printing out video metadata

  total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
  fps = video.get(cv2.CAP_PROP_FPS)
  print('FPS =', fps)
  print('Total Number of Frames=', total_frames)
  print()

  # Find SSIM metric between all frame pairs

  frame_num = 1
  ssim_values = []
  frame_times = []
  all_frame_pairs = []
  _, prev_image = video.read()

  print('Beginning to loop through frames...')

  while frame_num < total_frames:
    _, curr_image = video.read()

    frame_pair = '(' + str(frame_num) + ', ' + str(frame_num + 1) + ')'

    ssim_values.append(find_ssim(prev_image, curr_image))
    all_frame_pairs.append(frame_pair)
    frame_times.append((frame_num + 1) / fps);

    if (frame_num < 7) or (frame_num > total_frames - 5) or (frame_num % 100 == 0):
      print('Stored SSIM for frame pair', frame_pair)

    prev_image = curr_image
    frame_num += 1

  print('Process completed. Stored SSIM for all frame pairs.')
  print()

  # Outputting the graphs using Plotly

  plot1 = px.line(x=all_frame_pairs, y=ssim_values)
  plot2 = px.line(x=frame_times, y=ssim_values)
  plot3 = px.histogram(x=frame_times, y=ssim_values, nbins = 200)

  plot1.update_layout(
      xaxis=dict(title_text='Frame Pairs - (Frame 1, Frame 2)'),
      yaxis=dict(title_text='SSIM Metric', range=list([0, 1])),
      title_text='SSIM Metric for all Frame Pairs in Video',
      title_x=0.5
  )

  plot2.update_layout(
      xaxis=dict(dtick=2, title_text='Time of Second Frame in Frame Pair'),
      yaxis=dict(title_text='SSIM Metric', range=list([0, 1]))
  )

  plot3.update_layout(
      xaxis=dict(dtick=2, title_text='Time of Second Frame in Frame Pair'),
      yaxis=dict(title_text='SSIM Metric Sum')
  )

  return plot1, plot2, plot3

In [ ]:
video = cv2.VideoCapture('data/virat-kohli-nets.mp4')
plot1, plot2, plot3 = ssim_graphs(video)

plot1.show()
plot2.show()
plot3.show()

FPS = 25.0
Total Number of Frames= 2025.0

Beginning to loop through frames...
Stored SSIM for frame pair (1, 2)
Stored SSIM for frame pair (2, 3)
Stored SSIM for frame pair (3, 4)
Stored SSIM for frame pair (4, 5)
Stored SSIM for frame pair (5, 6)
Stored SSIM for frame pair (6, 7)
Stored SSIM for frame pair (100, 101)
Stored SSIM for frame pair (200, 201)
Stored SSIM for frame pair (300, 301)
Stored SSIM for frame pair (400, 401)
Stored SSIM for frame pair (500, 501)
Stored SSIM for frame pair (600, 601)
Stored SSIM for frame pair (700, 701)
Stored SSIM for frame pair (800, 801)
Stored SSIM for frame pair (900, 901)
Stored SSIM for frame pair (1000, 1001)
Stored SSIM for frame pair (1100, 1101)
Stored SSIM for frame pair (1200, 1201)
Stored SSIM for frame pair (1300, 1301)
Stored SSIM for frame pair (1400, 1401)
Stored SSIM for frame pair (1500, 1501)
Stored SSIM for frame pair (1600, 1601)
Stored SSIM for frame pair (1700, 1701)
Stored SSIM for frame pair (1800, 1801)
Stored SSIM for

In [ ]:
video = cv2.VideoCapture('data/batting-practice.mp4')
plot1, plot2, plot3 = ssim_graphs(video)

plot1.show()
plot2.show()
plot3.show()

FPS = 30.0
Total Number of Frames= 7777.0

Beginning to loop through frames...
Stored SSIM for frame pair (1, 2)
Stored SSIM for frame pair (2, 3)
Stored SSIM for frame pair (3, 4)
Stored SSIM for frame pair (4, 5)
Stored SSIM for frame pair (5, 6)
Stored SSIM for frame pair (6, 7)
Stored SSIM for frame pair (100, 101)
Stored SSIM for frame pair (200, 201)
Stored SSIM for frame pair (300, 301)
Stored SSIM for frame pair (400, 401)
Stored SSIM for frame pair (500, 501)
Stored SSIM for frame pair (600, 601)
Stored SSIM for frame pair (700, 701)
Stored SSIM for frame pair (800, 801)
Stored SSIM for frame pair (900, 901)
Stored SSIM for frame pair (1000, 1001)
Stored SSIM for frame pair (1100, 1101)
Stored SSIM for frame pair (1200, 1201)
Stored SSIM for frame pair (1300, 1301)
Stored SSIM for frame pair (1400, 1401)
Stored SSIM for frame pair (1500, 1501)
Stored SSIM for frame pair (1600, 1601)
Stored SSIM for frame pair (1700, 1701)
Stored SSIM for frame pair (1800, 1801)
Stored SSIM for

In [6]:
video = cv2.VideoCapture('data/mohit-cricket.mp4')
plot1, plot2, plot3 = ssim_graphs(video)

plot1.show()
plot2.show()
plot3.show()

FPS = 30.0
Total Number of Frames= 9090.0

Beginning to loop through frames...
Stored SSIM for frame pair (1, 2)
Stored SSIM for frame pair (2, 3)
Stored SSIM for frame pair (3, 4)
Stored SSIM for frame pair (4, 5)
Stored SSIM for frame pair (5, 6)
Stored SSIM for frame pair (6, 7)
Stored SSIM for frame pair (100, 101)
Stored SSIM for frame pair (200, 201)
Stored SSIM for frame pair (300, 301)
Stored SSIM for frame pair (400, 401)
Stored SSIM for frame pair (500, 501)
Stored SSIM for frame pair (600, 601)
Stored SSIM for frame pair (700, 701)
Stored SSIM for frame pair (800, 801)
Stored SSIM for frame pair (900, 901)
Stored SSIM for frame pair (1000, 1001)
Stored SSIM for frame pair (1100, 1101)
Stored SSIM for frame pair (1200, 1201)
Stored SSIM for frame pair (1300, 1301)
Stored SSIM for frame pair (1400, 1401)
Stored SSIM for frame pair (1500, 1501)
Stored SSIM for frame pair (1600, 1601)
Stored SSIM for frame pair (1700, 1701)
Stored SSIM for frame pair (1800, 1801)
Stored SSIM for